<a href="https://colab.research.google.com/github/Dededon/COVID_Twitter/blob/main/Embedding%20Controversy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from itertools import combinations
import pandas as pd
import numpy as np
import numba as nb

In [ ]:
def nodeSameCluster(nodes):
    return combinations(nodes, 2)

@nb.njit(fastmath=True)
def norm(array1, array2):
    s = 0
    for i in range(array1.shape[0]):
        s += (array1[i] - array2[i])**2
    return s**0.5

In [ ]:
df = pd.read_csv("drive/MyDrive/HKUST/Comp 4641 Project/mask_co_hashtag_t2_cluster.csv")[["node", "cluster", "x", "y"]]
df.head()

,node,cluster,x,y
0,sdinny14,0,6.693446,9.168682
1,erinstreeterNAM,0,14.893178,7.216660
2,vinylcook,0,7.128399,7.869806
3,ShirtStru,0,9.992977,2.943196
4,brose2288,1,6.757268,-5.555685


In [ ]:
df.describe()

,cluster,x,y
count,3425.000000,3425.000000,3425.000000
mean,0.378686,8.251950,5.910208
std,0.657417,7.010506,7.832296
min,0.000000,-10.928577,-11.752399
25%,0.000000,5.651793,2.193761
50%,0.000000,7.913805,6.349135
75%,1.000000,12.210897,10.690126
max,2.000000,26.093376,22.022476


In [ ]:
n_cluster = int(df.describe().loc["max", "cluster"] + 1)

In [ ]:
clusterNodes = list()
for i in range(n_cluster):
    clusterNodes.append(df[df["cluster"] == i]["node"].values.tolist())

In [ ]:
nodes_pos = {df.loc[i]["node"]: df.loc[i][["x", "y"]].values.astype(np.float32) for i in range(len(df))}

In [ ]:
def getInterClusterDistance(nodes):
    pairs = nodeSameCluster(nodes)
    sum_dist = 0
    cnt = 0
    for p in pairs:
        pos1 = nodes_pos[p[0]]
        pos2 = nodes_pos[p[1]]
        sum_dist += norm(pos1, pos2)
        cnt += 1
    return sum_dist / cnt
interDist = list()
for nodesList in clusterNodes:
    interDist.append(getInterClusterDistance(nodesList))
interDist

[8.605443231132512, 8.426646268931133, 7.41378245653378]

In [ ]:
def getIntraClusterDistance(nodes1, nodes2):
    sum_dist = 0
    for n1 in nodes1:
        pos1 = nodes_pos[n1]
        for n2 in nodes2:
            pos2 = nodes_pos[n2]
            sum_dist += norm(pos1, pos2)
    return sum_dist / (len(nodes1) * len(nodes2))

ECscore = list()
for pairs in combinations(range(n_cluster), 2):
    ECscore.append(1 - (interDist[pairs[0]] + interDist[pairs[1]]) / ( 2 * getIntraClusterDistance(clusterNodes[pairs[0]], clusterNodes[pairs[1]])))
ECscore

[0.5095375250449774, 0.5541058177394484, 0.6870984339410499]

In [ ]:
np.mean(ECscore)

0.5835805922418252